<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

The following functions define a system of nonlinear equations and its
jacobian.


In [ ]:
import numpy as np
def fun(x):
    return [x[0]  + 0.5 * (x[0] - x[1])**3 - 1.0,
            0.5 * (x[1] - x[0])**3 + x[1]]

In [ ]:
def jac(x):
    return np.array([[1 + 1.5 * (x[0] - x[1])**2,
                      -1.5 * (x[0] - x[1])**2],
                     [-1.5 * (x[1] - x[0])**2,
                      1 + 1.5 * (x[1] - x[0])**2]])

A solution can be obtained as follows.


In [ ]:
from scipy import optimize
sol = optimize.root(fun, [0, 0], jac=jac, method='hybr')
sol.x

array([ 0.8411639,  0.1588361])

**Large problem**

Suppose that we needed to solve the following integrodifferential
equation on the square $[0,1]\times[0,1]$:

$$ \nabla^2 P = 10 \left(\int_0^1\int_0^1\cosh(P)\,dx\,dy\right)^2 $$
with $P(x,1) = 1$ and $P=0$ elsewhere on the boundary of
the square.

The solution can be found using the ``method='krylov'`` solver:


In [ ]:
from scipy import optimize
# parameters
nx, ny = 75, 75
hx, hy = 1./(nx-1), 1./(ny-1)

In [ ]:
P_left, P_right = 0, 0
P_top, P_bottom = 1, 0

In [ ]:
def residual(P):
   d2x = np.zeros_like(P)
   d2y = np.zeros_like(P)

   d2x[1:-1] = (P[2:]   - 2*P[1:-1] + P[:-2]) / hx/hx
   d2x[0]    = (P[1]    - 2*P[0]    + P_left)/hx/hx
   d2x[-1]   = (P_right - 2*P[-1]   + P[-2])/hx/hx

   d2y[:,1:-1] = (P[:,2:] - 2*P[:,1:-1] + P[:,:-2])/hy/hy
   d2y[:,0]    = (P[:,1]  - 2*P[:,0]    + P_bottom)/hy/hy
   d2y[:,-1]   = (P_top   - 2*P[:,-1]   + P[:,-2])/hy/hy

   return d2x + d2y - 10*np.cosh(P).mean()**2

In [ ]:
guess = np.zeros((nx, ny), float)
sol = optimize.root(residual, guess, method='krylov')
print('Residual: %g' % abs(residual(sol.x)).max())

Residual: 5.7972e-06  # may vary

In [ ]:
import matplotlib.pyplot as plt
x, y = np.mgrid[0:1:(nx*1j), 0:1:(ny*1j)]
plt.pcolormesh(x, y, sol.x, shading='gouraud')
plt.colorbar()
plt.show()